# 0. Install and Import Dependencies

In [1]:
import mediapipe as mp
import cv2
import numpy as np

In [2]:
mp_drawing = mp.solutions.drawing_utils # helpers for drawing
mp_pose = mp.solutions.pose

In [3]:
VIDEO_PATH = 'classification/videos/bicep-curl-right1.mp4'
EXPORT_PATH = 'classification/coords/coords-new.csv'
MODEL_PATH = 'classification/models/bicep-curl-new.pkl'

# X. Make Some Detections

In [ ]:
cap = cv2.VideoCapture(0)

# Initiate holistic model
with mp_pose.Pose(min_detection_confidence= 0.5, min_tracking_confidence= 0.5) as pose:
    try:
        while cap.isOpened():
            ret, image = cap.read()

            if ret is False:
                print("Error Camera Not Found.")
            if image is None:
                print("Ignoring empty camera frame.")
                break

            # Recolor Feed
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            # Make Detections
            results = pose.process(image)

            # Recolor image back to BGR for rendering
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                      mp_drawing.DrawingSpec(color= (245, 117, 66),
                                                             thickness= 2,
                                                             circle_radius= 4),
                                      mp_drawing.DrawingSpec(color= (245, 66, 230),
                                                             thickness= 2,
                                                             circle_radius= 2)
                                     )
            
            # test code
            
#             test = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten().tolist()
#             test.insert(0, 0)

#             left_shoulder = (test[45], test[46])
#             right_shoulder = (test[49], test[50])
#             left_elbow = (test[53], test[54])
#             right_elbow = (test[57], test[58])
#             left_wrist = (test[61], test[62])
#             right_wrist = (test[65], test[66])
#             left_hip = (test[93], test[94])
#             right_hip = (test[97], test[98])
#             left_knee = (test[101], test[102])
#             right_knee = (test[105], test[106])

#             left_elbow_ang = angle(left_shoulder, left_elbow, left_wrist)
#             right_elbow_ang = angle(right_shoulder, right_elbow, right_wrist)

#             left_lean_ang = angle(left_shoulder, left_hip, left_knee)
#             right_lean_ang = angle(right_shoulder, right_hip, right_knee)

#             left_shoulder_ang = angle(left_elbow, left_shoulder, left_hip)
#             right_shoulder_ang = angle(right_elbow, right_shoulder, right_hip)

#             image = draw_text(image, f'Elbow {int(left_elbow_ang)}', (50, 100))
#             # image = draw_text(image, f'Lean {int(left_lean_ang)}', (50, 150))
#             image = draw_text(image, f'Shoulder {int(left_shoulder_ang)}', (50, 150))
            
            # end test code

            cv2.imshow('TEST', image)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

    except Exception as e:
        cap.release()
        cv2.destroyAllWindows()
            
cap.release()
cv2.destroyAllWindows()

# 1. Save Video

In [ ]:
cap = cv2.VideoCapture(0)

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
videoWriter = cv2.VideoWriter('bicep-curl.mp4', cv2.VideoWriter_fourcc('P','I','M','1'), fps, (width, height))

while cap.isOpened():
    ret, frame = cap.read()

    try:
        cv2.imshow('Bicep-Curl', frame)
        videoWriter.write(frame)
    except Exception as e:
        break

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
videoWriter.release()
cv2.destroyAllWindows()

# 2. Capture Landmarks & Export to CSV

In [4]:
import csv
import os
from matplotlib import pyplot as plt
from math import atan2, degrees

Create `CSV` file

In [5]:
landmarks = ['class']

for val in range(0, 32 +1):
    landmarks += [f'x{val}', f'y{val}', f'z{val}', f'v{val}']

# additional features
landmarks += ['left_elbow_ang', 'right_elbow_ang', 
              'left_lean_ang', 'right_lean_ang', 
              'left_shoulder_ang', 'right_shoulder_ang',
              'left_elbow_hip_dist', 'right_elbow_hip_dist']

In [6]:
with open(EXPORT_PATH, mode= 'w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

## 2.0 Define Necessary Functions

In [ ]:
# without class the points will be ...
POSE = {
  "nose": (0, 1),
  "left_eye_inner": (4, 5),
  "left_eye": (8, 9),
  "left_eye_outer": (12, 13),
  "right_eye_inner": (16, 17),
  "right_eye": (20, 21),
  "right_eye_outer": (24, 25),
  "left_ear": (28, 29),
  "right_ear": (32, 33),
  "mouth_left": (36, 37),
  "mouth_right": (40, 41),
  "left_shoulder": (44, 45),
  "right_shoulder": (48, 49),
  "left_elbow": (52, 53),
  "right_elbow": (56, 57),
  "left_wrist": (60, 61),
  "right_wrist": (64, 65),
  "left_pinky": (68, 69),
  "right_pinky": (72, 73),
  "left_index": (76, 77),
  "right_index": (80, 81),
  "left_thumb": (84, 85),
  "right_thumb": (88, 89),
  "left_hip": (92, 93),
  "right_hip": (96, 97),
  "left_knee": (100, 101),
  "right_knee": (104, 105),
  "left_ankle": (108, 109),
  "right_ankle": (112, 113),
  "left_heel": (116, 117),
  "right_heel": (120, 121),
  "left_foot_index": (124, 125),
  "right_foot_index": (128, 129),
}

In [7]:
def angle(a, b, c):
    """ Calculate angle between two lines """
    if any(point == (0, 0) for point in [a, b, c]):
        return 0
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    if angle > 180.0:
        angle = 360 - angle
    return angle

In [8]:
def draw_text(img, text, pos, font_scale=None, font_color=None, font_width=None, line_type=None):
    """ Draw a intersection point in an image """
    font = cv2.FONT_HERSHEY_PLAIN    
    if font_scale is None:
        font_scale = 1.5
    if font_color is None:
        font_color = (0, 255, 0)
    if font_width is None:
        font_width = 3
    if line_type is None:
        line_type = cv2.LINE_AA
        
    cv2.putText(img, text, pos, font, font_scale, font_color, font_width, line_type)
    return img

In [9]:
def export_landmark(results, action):
    try:
        keypoints = [
            action,
            *[res.x for res in results.pose_landmarks.landmark],
            *[res.y for res in results.pose_landmarks.landmark],
            *[res.z for res in results.pose_landmarks.landmark],
            *[res.visibility for res in results.pose_landmarks.landmark],
        ]
        
        left_shoulder = (keypoints[45], keypoints[46])
        right_shoulder = (keypoints[49], keypoints[50])
        left_elbow = (keypoints[53], keypoints[54])
        right_elbow = (keypoints[57], keypoints[58])
        left_wrist = (keypoints[61], keypoints[62])
        right_wrist = (keypoints[65], keypoints[66])
        left_hip = (keypoints[93], keypoints[94])
        right_hip = (keypoints[97], keypoints[98])
        left_knee = (keypoints[101], keypoints[102])
        right_knee = (keypoints[105], keypoints[106])
        
        left_elbow_ang = angle(left_shoulder, left_elbow, left_wrist)
        right_elbow_ang = angle(right_shoulder, right_elbow, right_wrist)
        left_lean_ang = angle(left_shoulder, left_hip, left_knee)
        right_lean_ang = angle(right_shoulder, right_hip, right_knee)
        left_shoulder_ang = angle(left_elbow, left_shoulder, left_hip)
        right_shoulder_ang = angle(right_elbow, right_shoulder, right_hip)
        left_elbow_hip_dist = dist_x(left_hip, left_elbow)
        right_elbow_hip_dist = dist_x(right_hip, right_elbow)
        
        keypoints += [
            left_elbow_ang,
            right_elbow_ang,
            left_lean_ang,
            right_lean_ang,
            left_shoulder_ang,
            right_shoulder_ang,
            left_elbow_hip_dist,
            right_elbow_hip_dist,
        ]
        
        with open(EXPORT_PATH, mode='a', newline='') as f:
            csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            csv_writer.writerow(keypoints)
        
        return [
            f'ElbowAng(L): {int(left_elbow_ang)}',
            f'LeanAng(L): {int(left_lean_ang)}',
            f'ShoulderAng(L): {int(left_shoulder_ang)}',
            f'ElbowDist(L): {int(left_elbow_hip_dist)}'
        ]
    
    except Exception as e:
        print(f'Error in export_landmark: {e}')

In [10]:
def dist_x(a, b):
    return (b[0] - a[0]) * 100

In [ ]:
def dist_xy(a, b):
    diff_point1 = (a[0] - b[0]) ** 2
    diff_point2 = (a[1] - b[1]) ** 2
    return ((diff_point1 + diff_point2) ** 0.5) * 100

In [ ]:
# left arm
angle_between(keypoints[11], keypoints[13], keypoints[15])

In [ ]:
# right arm
angle_between(keypoints[12], keypoints[14], keypoints[16])

## 2.1 Train from Right Side

In [11]:
cap = cv2.VideoCapture(VIDEO_PATH)

# Initiate holistic model
with mp_pose.Pose(min_detection_confidence= 0.5, min_tracking_confidence= 0.5) as pose:
    
    right_elbow_ang, right_lean_ang, right_shoulder_ang, right_elbow_hip_dist = 0, 0, 0, 0
    features = [f'ElbowAng(R): {right_elbow_ang}', f'LeanAng(R): {right_lean_ang}', f'ShoulderAng(R): {right_shoulder_ang}', f'ElbowDist(R): {right_elbow_hip_dist}']
    
    try:
        while cap.isOpened():
            ret, image = cap.read()
            
            # Handle end of video
            if image is None:
                print("Ignoring empty camera frame.")
                break

            if ret is False:
                print("Error File Not Found.")
                break

            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            results = pose.process(image)

            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                      mp_drawing.DrawingSpec(color=(245, 117, 66),
                                                             thickness=2,
                                                             circle_radius=4),
                                      mp_drawing.DrawingSpec(color=(245, 66, 230),
                                                             thickness=2,
                                                             circle_radius=2))

            # Wait for user input
            k = cv2.waitKey(1)
            if k == ord('a'):
                action = 'correct'
            elif k == ord('s'):
                action = 'error1'
            elif k == ord('w'):
                action = 'error2'
            else:
                action = None

            # Display feature values on image
            if action is not None:
                features = export_landmark(results, action)

            image = draw_text(image, str(features[0]), (40, 550))
            image = draw_text(image, str(features[1]), (40, 600))
            image = draw_text(image, str(features[2]), (40, 575))
            image = draw_text(image, str(features[3]), (40, 625))
            
            cv2.imshow('Bicep Training - Right Side', image)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
                
    except Exception as e:
        print(f'Looping through frames Exception: {e}')
    finally:
        # Release video capture object and close windows
        cap.release()
        cv2.destroyAllWindows()

## 2.2 Train from Left Side

In [ ]:
cap = cv2.VideoCapture(VIDEO_PATH)

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    
    left_elbow_ang, left_lean_ang, left_shoulder_ang, left_elbow_hip_dist = 0, 0, 0, 0  # Initialize features
    features = [f'ElbowAng(L): {left_elbow_ang}', f'LeanAng(L): {left_lean_ang}', f'ShoulderAng(L): {left_shoulder_ang}', f'ElbowDist(L): {left_elbow_hip_dist}']
    
    try:
        while cap.isOpened():
            ret, image = cap.read()

            # Handle end of video
            if image is None:
                print("Ignoring empty camera frame.")
                break
                
            if not ret:
                print("Error File Not Found.")
                break

            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            results = pose.process(image)

            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                      mp_drawing.DrawingSpec(color=(245, 117, 66),
                                                             thickness=2,
                                                             circle_radius=4),
                                      mp_drawing.DrawingSpec(color=(245, 66, 230),
                                                             thickness=2,
                                                             circle_radius=2))

            # Wait for user input
            k = cv2.waitKey(1)
            if k == ord('a'):
                action = 'correct'
            elif k == ord('s'):
                action = 'error1'
            elif k == ord('w'):
                action = 'error2'
            else:
                action = None

            # Display feature values on image
            if action is not None:
                features = export_landmark(results, action)

            image = draw_text(image, str(features[0]), (40, 520))
            image = draw_text(image, str(features[1]), (40, 580))
            image = draw_text(image, str(features[2]), (40, 550))
            image = draw_text(image, str(features[3]), (40, 610))

            cv2.imshow('Bicep Training - Left Side', image)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

    except Exception as e:
        print(f'Looping through frames Exception: {e}')
    finally:
        # Release video capture object and close windows
        cap.release()
        cv2.destroyAllWindows()

# 3. Train Custom Model Using Scikit Learn

## 3.1 Read in Collected Data and Process

In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split

C:\Users\Galileo\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [16]:
df = pd.read_csv(EXPORT_PATH)

In [17]:
df.head()

,class,x0,y0,z0,v0,x1,y1,z1,v1,x2,...,z32,v32,left_elbow_ang,right_elbow_ang,left_lean_ang,right_lean_ang,left_shoulder_ang,right_shoulder_ang,left_elbow_hip_dist,right_elbow_hip_dist
0,correct,0.561922,0.259988,-0.269829,0.997965,0.540063,0.237290,-0.231917,0.998634,0.538414,...,-0.029422,0.268631,91.146849,87.285568,184.904018,183.585602,0.029108,6.986046,-1.361185,-5.660853
1,correct,0.559385,0.258238,-0.251098,0.997848,0.535848,0.235986,-0.213423,0.998566,0.534020,...,-0.022837,0.254273,102.362295,91.814375,184.805063,183.467658,1.118270,8.151278,-1.325235,-6.090540
2,correct,0.557762,0.256512,-0.244626,0.997783,0.534479,0.234822,-0.205829,0.998497,0.532894,...,-0.016710,0.242313,114.938288,94.678389,187.152177,183.366366,0.963891,9.516275,-1.392528,-6.521660
3,correct,0.556221,0.253643,-0.256800,0.997548,0.533388,0.233141,-0.218037,0.998240,0.531971,...,0.021587,0.230560,122.573191,96.378917,187.659913,182.458472,0.078111,10.579317,-1.594070,-6.763697
4,correct,0.554711,0.251110,-0.261652,0.997488,0.531336,0.231245,-0.223392,0.998122,0.529772,...,0.016959,0.222095,119.695699,98.125001,184.462528,181.734917,0.149334,12.246157,-1.618409,-7.152286


In [18]:
df.shape

(4646, 141)

In [39]:
df[df['class'] == 'correct']

,class,x0,y0,z0,v0,x1,y1,z1,v1,x2,...,z32,v32,left_elbow_ang,right_elbow_ang,left_lean_ang,right_lean_ang,left_shoulder_ang,right_shoulder_ang,left_elbow_hip_dist,right_elbow_hip_dist
0,correct,0.561922,0.259988,-0.269829,0.997965,0.540063,0.237290,-0.231917,0.998634,0.538414,...,-0.029422,0.268631,91.146849,87.285568,184.904018,183.585602,0.029108,6.986046,-1.361185,-5.660853
1,correct,0.559385,0.258238,-0.251098,0.997848,0.535848,0.235986,-0.213423,0.998566,0.534020,...,-0.022837,0.254273,102.362295,91.814375,184.805063,183.467658,1.118270,8.151278,-1.325235,-6.090540
2,correct,0.557762,0.256512,-0.244626,0.997783,0.534479,0.234822,-0.205829,0.998497,0.532894,...,-0.016710,0.242313,114.938288,94.678389,187.152177,183.366366,0.963891,9.516275,-1.392528,-6.521660
3,correct,0.556221,0.253643,-0.256800,0.997548,0.533388,0.233141,-0.218037,0.998240,0.531971,...,0.021587,0.230560,122.573191,96.378917,187.659913,182.458472,0.078111,10.579317,-1.594070,-6.763697
4,correct,0.554711,0.251110,-0.261652,0.997488,0.531336,0.231245,-0.223392,0.998122,0.529772,...,0.016959,0.222095,119.695699,98.125001,184.462528,181.734917,0.149334,12.246157,-1.618409,-7.152286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2871,correct,0.565547,0.463755,-0.080202,0.998117,0.556966,0.446149,-0.057167,0.997959,0.554687,...,-0.094405,0.387444,96.994956,85.040168,172.548727,164.916312,23.250984,4.351472,-5.411097,1.292744
2872,correct,0.570574,0.463762,-0.072405,0.998228,0.561160,0.446229,-0.049414,0.998082,0.558665,...,-0.064554,0.391178,95.776169,89.778274,170.899246,164.228427,20.773302,8.025016,-4.838637,2.121601
2873,correct,0.574383,0.463865,-0.080072,0.998340,0.565628,0.447052,-0.056416,0.998200,0.563615,...,-0.120682,0.398591,98.268765,94.482184,169.012057,161.805444,19.714048,14.686912,-4.435566,3.690943
2874,correct,0.579698,0.464151,-0.053293,0.998422,0.570055,0.448138,-0.030909,0.998302,0.568075,...,-0.161614,0.398642,103.514851,101.253528,166.934481,159.568376,21.391438,25.511235,-4.629698,6.192911


In [41]:
print('Correct = {}'.format(df[df['class'] == 'correct'].shape[0]), 
      '\nLeaning: {}'.format(df[df['class'] == 'error1'].shape[0]), 
      '\nElbow Out: {}'.format(df[df['class'] == 'error2'].shape[0]))

Correct = 1221 
Leaning: 1510 
Elbow Out: 1915


In [42]:
X = df.drop('class', axis= 1) 
y = df['class']

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify= y, test_size= 0.3, random_state= 7)

In [44]:
y_test

1621    correct
2379     error1
3518     error2
2853    correct
2453     error2
         ...   
1530    correct
3513     error2
2209     error1
3043     error1
383     correct
Name: class, Length: 1394, dtype: object

## 3.2 Train Machine Learning Classification Model

In [45]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [46]:
pipelines = {
    'lr': make_pipeline(StandardScaler(), LogisticRegression()),
    'rc': make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf': make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb': make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

In [47]:
fit_models = {}

for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model

In [48]:
fit_models

{'lr': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('logisticregression', LogisticRegression())]),
 'rc': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('ridgeclassifier', RidgeClassifier())]),
 'rf': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())]),
 'gb': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('gradientboostingclassifier', GradientBoostingClassifier())])}

In [49]:
fit_models['rc'].predict(X_test)

array(['correct', 'error1', 'error2', ..., 'error1', 'error1', 'correct'],
      dtype='<U7')

## 3.3 Evaluate and Serialize Model

In [50]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
import pickle

In [51]:
for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algo,
         accuracy_score(y_test.values, yhat),
         precision_score(y_test.values, yhat, average= "micro", labels= "correct"),
         recall_score(y_test.values, yhat, average= "micro", labels= "correct"),
         recall_score(y_test.values, yhat, average= "micro", labels= "correct")
        )

lr 0.9971305595408895 0.9971305595408895 0.9971305595408895 0.9971305595408895
rc 0.9971305595408895 0.9971305595408895 0.9971305595408895 0.9971305595408895
rf 0.9992826398852224 0.9992826398852224 0.9992826398852224 0.9992826398852224
gb 0.9992826398852224 0.9992826398852224 0.9992826398852224 0.9992826398852224


In [52]:
yhat = fit_models['rf'].predict(X_test)

In [53]:
yhat[ :10]

array(['correct', 'error1', 'error2', 'correct', 'error2', 'error1',
       'error2', 'error2', 'error1', 'error2'], dtype=object)

In [54]:
with open(MODEL_PATH, 'wb') as f:
    pickle.dump(fit_models['rf'], f)

# 4. Make Detections with Model

In [59]:
def get_features(keypoint):
    
    left_shoulder = (keypoint[44], keypoint[45])
    right_shoulder = (keypoint[48], keypoint[49])
    left_elbow = (keypoint[52], keypoint[53])
    right_elbow = (keypoint[56], keypoint[57])
    left_wrist = (keypoint[60], keypoint[61])
    right_wrist = (keypoint[64], keypoint[65])
    left_hip = (keypoint[92], keypoint[93])
    right_hip = (keypoint[96], keypoint[97])
    left_knee = (keypoint[100], keypoint[101])
    right_knee = (keypoint[104], keypoint[105])

    left_elbow_ang = angle(left_shoulder, left_elbow, left_wrist)
    right_elbow_ang = angle(right_shoulder, right_elbow, right_wrist)
    left_lean_ang = angle(left_shoulder, left_hip, left_knee)
    right_lean_ang = angle(right_shoulder, right_hip, right_knee)
    left_shoulder_ang = angle(left_elbow, left_shoulder, left_hip)
    right_shoulder_ang = angle(right_elbow, right_shoulder, right_hip)
    left_elbow_hip_dist = dist_x(left_hip, left_elbow)
    right_elbow_hip_dist = dist_x(right_hip, right_elbow)
    
    return [left_elbow_ang, right_elbow_ang, left_lean_ang, right_lean_ang, left_shoulder_ang, right_shoulder_ang, left_elbow_hip_dist, right_elbow_hip_dist]

In [60]:
def append_features(row, features):
    for feature in features:
        row.append(feature)
        
    return row

In [61]:
def draw_text_background(img, text, pos, font_scale, font_thickness, txt_color=None, bckg_color=None, line_type= None):
    font = cv2.FONT_HERSHEY_SIMPLEX
    if txt_color is None:
        txt_color= (0, 0, 0)
    if bckg_color is None:
        bckg_color=(255, 255, 255)
    if line_type is None:
        line_type = cv2.LINE_AA
    
    text_size, _ = cv2.getTextSize(text, font, font_scale, font_thickness)
    text_w, text_h = text_size
    rect_w, rect_h = text_w + 10, text_h + 15
    rect_x, rect_y = pos[0] - 5, pos[1] - text_h - 5

    cv2.rectangle(img, (rect_x, rect_y), (rect_x + rect_w, rect_y + rect_h), bckg_color, -1)
    cv2.putText(img, text, pos, font, font_scale, txt_color, font_thickness, line_type)

    return img

In [62]:
with open(MODEL_PATH, 'rb') as f:
    pickle.load(f)

In [63]:
cap = cv2.VideoCapture(VIDEO_PATH)

# Initiate holistic model
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    try:
        while cap.isOpened():
            ret, image = cap.read()

            # Handle end of video
            if image is None:
                print("Ignoring empty camera frame.")
                break
                
            if not ret:
                print("Error File Not Found.")
                break

            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            results = pose.process(image)

            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                      mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=4),
                                      mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

            try:
                row = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten().tolist()
                features = get_features(row)
                row = append_features(row, features)
                X = pd.DataFrame([row], columns=landmarks[1:])

                body_language_prob = model.predict_proba(X)[0]
                body_language_class = np.argmax(body_language_prob)

                state = 'Unknown'
                if body_language_prob[body_language_class] >= 0.7:
                    if body_language_class == 0:
                        state = 'Correct'
                    elif body_language_class == 1:
                        state = 'Elbow Out Of Place'
                    elif body_language_class == 2:
                        state = 'Leaning'
                
                # Display Class and Probability
                font_scale = 1
                font_thickness = 2
                font_color = (0, 0, 255)
                
                class_txt = f'CLASS: {state}'
                class_txt_pos = (10, 30)
                image = draw_text_background(image, class_txt, class_txt_pos, font_scale, font_thickness)
    
                prob_txt = 'PROB: {:.2f}'.format(body_language_prob[body_language_class])
                prob_txt_pos = (10, 70)
                image = draw_text_background(image, prob_txt, prob_txt_pos, font_scale, font_thickness)

            except Exception as e:
                print(f'Class Prediction Exception: {e}')

            cv2.imshow('Testing Biceps Model', image)
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        
    except Exception as e:
        print(f'Looping through frames Exception: {e}')
    finally:
        # Release video capture object and close windows
        cap.release()
        cv2.destroyAllWindows()

cap.release()
cv2.destroyAllWindows()

In [ ]:
cap = cv2.VideoCapture(0)
counter = 0
current_stage = ''
state = ''

# Initiate holistic model
with mp_pose.Pose(min_detection_confidence= 0.5, min_tracking_confidence= 0.5) as pose:
    
    try:
        while cap.isOpened():
            ret, image = cap.read()

            if ret is False:
                    print("Error File Not Found.")
            if image is None:
                print("Ignoring empty camera frame.")
                break     

            # Recolor Feed
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
            # Make Detections
            results = pose.process(image)

            # Recolor image back to BGR for rendering
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                      mp_drawing.DrawingSpec(color= (245, 117, 66),
                                                             thickness= 2,
                                                             circle_radius= 4),
                                      mp_drawing.DrawingSpec(color= (245, 66, 230),
                                                             thickness= 2,
                                                             circle_radius= 2)
                                     )

            try:
                row = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten().tolist()
                features = get_features(row)
                row = append_features(row, features)
                X = pd.DataFrame([row], columns= landmarks[1:])

                
                body_language_class = model.predict(X)[0]
                body_language_prob = model.predict_proba(X)[0]
                # print(body_language_class, body_language_prob)
                correct = False; error1 = False; error2 = False

                if body_language_class == 'correct' and body_language_prob[body_language_prob.argmax()] >= .7:
                    correct = True
                    error1 = False
                    error2 = False
                elif body_language_class == 'error1' and body_language_prob[body_language_prob.argmax()] >= .7:
                    correct = False
                    error1 = True
                    error2 = False
                elif body_language_class == 'error2' and body_language_prob[body_language_prob.argmax()] >= .7:
                    correct = False
                    error1 = False
                    error2 = True

                if (correct):
                    state = 'Correct'
                elif (error1):
                    state = 'Elbow Out Of Place'
                elif (error2):
                    state = 'Leaning'

                # elif current_stage = 'down' and body_language_class == 'up' and body_language_prob[body_language_prob.argmax()] >= .7:
                #     current_stage = 'up'
                #     counter += 1

                counter = 0


                    # print(current_stage)

                # Get status box
                #cv2.rectangle(image, (0,0), (250,60), (245,117,16), -1)

                # Display Class
                cv2.putText(image, 
                            'CLASS',
                            (0,0),
                            cv2.FONT_HERSHEY_SIMPLEX,
                            0.5,
                            (0,0,0),
                            1,
                            cv2.LINE_AA
                           )

                # cv2.putText(image, 
                #             body_language_class(' ')[0],
                #             (90,40),
                #             cv2.FONT_HERSHEY_SIMPLEX,
                #             1,
                #             (255,255,255),
                #             2,
                #             cv2.LINE_AA
                #            )
                
                #Display Propability
                cv2.putText(image, 
                            'PROB',
                            (15,12),
                            cv2.FONT_HERSHEY_SIMPLEX,
                            0.5,
                            (0,0,0),
                            1,
                            cv2.LINE_AA
                           )
                cv2.putText(image, 
                            str(round(body_language_prob[np.argmax(body_language_prob)], 2)),
                            (10,40),
                            cv2.FONT_HERSHEY_SIMPLEX,
                            1,
                            (0,0,0),
                            2,
                            cv2.LINE_AA
                           )

                #Display Count
                cv2.putText(image, 
                            'CLASS',
                            (180,12),
                            cv2.FONT_HERSHEY_SIMPLEX,
                            0.5,
                            (0,0,0),
                            1,
                            cv2.LINE_AA
                           )
                cv2.putText(image, 
                            str(state),
                            (175,40),
                            cv2.FONT_HERSHEY_SIMPLEX,
                            1,
                            (0,0,255),
                            2,
                            cv2.LINE_AA
                           )


            except Exception as e:
                print(f'error {e}')

            cv2.imshow('Lucio', image)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
            
    except Exception as e:
        print(f'error {e}')
        cap.release()
        cv2.destroyAllWindows()
        
cap.release()
cv2.destroyAllWindows()